# LangChain: Memory

https://learn.deeplearning.ai/courses/langchain/lesson/3/memory

## Outline
* **ConversationBufferMemory:** We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.


![memory](resources/memory.png)


* **ConversationBufferWindowMemory:** Set the number of last K messages to save in Message History
* **ConversationTokenBufferMemory:** Limit the number of token in Message History
* **ConversationSummaryMemory:** Make a summarization of Message History
 
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

Importantly, you will want to do this **BEFORE** the prompt template but **AFTER** you load previous messages from Message History.

In [3]:
import warnings
import os
from IPython.display import display, Markdown  # to see better the output text
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

warnings.filterwarnings('ignore')

## ConversationBufferMemory

In [20]:
from langchain_openai import ChatOpenAI

from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory

llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.0, model=llm_model)


memory = ConversationBufferMemory()


conversation = ConversationChain(
    llm=llm,

    memory=memory,

    verbose=True

)

In [21]:
conversation.predict(input="Hi, my name is Manuel")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Manuel
AI:

> Finished chain.


"Hello Manuel! It's nice to meet you. How can I assist you today?"

In [22]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Manuel
AI: Hello Manuel! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

In [23]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Manuel
AI: Hello Manuel! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Manuel.'

In [24]:
print(memory.buffer)

Human: Hi, my name is Manuel
AI: Hello Manuel! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Manuel.


In [28]:
print(memory.load_memory_variables({})["history"])

Human: Hi, my name is Manuel
AI: Hello Manuel! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Manuel.


In [29]:
memory = ConversationBufferMemory()

In [30]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [31]:
print(memory.buffer)

Human: Hi
AI: What's up


## ConversationBufferWindowMemory

In [36]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)               


In [37]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [38]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [39]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [40]:
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [41]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to ask?"

## ConversationTokenBufferMemory

In [43]:
#!pip install tiktoken

In [44]:
from langchain.memory import ConversationTokenBufferMemory
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [52]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [50]:
print(memory.load_memory_variables({})["history"])

Human: Chatbots are what?
AI: Charming!


## ConversationSummaryMemory

In [88]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [89]:
Markdown(memory.buffer)

System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.

## `ConversationChain` is deprecated in langchain ^0.2 in favor of `RunnableWithMessageHistory`.
Please refer to this tutorial for more detail: https://python.langchain.com/v0.2/docs/tutorials/chatbot/

A key part here is the function we pass into as the `get_session_history`. This function is expected to take in a `session_id` and return a Message History object. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain.


In [58]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [59]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello Bob! How can I assist you today?'

If we set a diferent `session_id` the chatbot can't remember the name

In [60]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I don't have access to that information."

In [61]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob.'

In [67]:
message_histoy: BaseChatMessageHistory = get_session_history("abc2")

In [73]:
message_histoy.messages

[HumanMessage(content="Hi! I'm Bob"),
 AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-46aa395d-a20a-4bd8-865d-4fe0fbba2062-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22}),
 HumanMessage(content="What's my name?"),
 AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f29e433-184b-469e-9e3f-cf99504390ac-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})]

## Limit the number of MessageHistory using `RunnableWithMessageHistory`
LangChain comes with a few built-in helpers for managing a list of messages (alterantives for ConversationBufferWindowMemory, ConversationTokenBufferMemory, ConversationSummaryMemory).
 
See https://python.langchain.com/v0.2/docs/how_to/#messages